## Integrantes

Angello Turano RM: 556511

Cauã Sanches RM:558317

Leonardo Bianchi RM:558576




## Para prever o risco de queimadas com base em dados ambientais coletados por sensores IoT, desenvolvemos uma solução de machine learning com as seguintes etapas:

1. Geração de Dados Sintéticos
Como ainda não há dados reais de sensores disponíveis, simulamos um conjunto com 1.000 registros, contendo as seguintes variáveis ambientais:

temperatura (°C)

umidade (%)

fumaca (ppm)

vento (km/h)

Também criamos uma variável alvo chamada risco_queimada, com valor 1 (com risco) ou 0 (sem risco), baseado na hipótese:

Se temperatura > 35°C e umidade < 30% e fumaca > 120 ppm, então há risco de queimada.

2. Exploração de Dados (EDA)
Realizamos uma análise exploratória para entender melhor os dados:

Visualização dos dados com histogramas;

Estatísticas descritivas (média, desvio padrão, etc.);

Comparação das médias das variáveis entre os casos com e sem risco;

Verificação do equilíbrio entre as classes (0 e 1).

3. Preparação dos Dados
Selecionamos as variáveis de entrada (temperatura, umidade, fumaca, vento) e a variável alvo (risco_queimada). Dividimos os dados em:

Treinamento (70%)

Teste (30%)

Mantivemos a proporção de classes usando stratify no train_test_split.

4. Treinamento do Modelo
Utilizamos o algoritmo Random Forest, que é robusto, não-linear e funciona bem com dados tabulares e variáveis correlacionadas.

5. Avaliação do Modelo
O modelo foi avaliado com:

Matriz de confusão

Relatório de classificação (precisão, recall, F1-score)

Validação cruzada (5-fold) para garantir robustez

6. Importância das Variáveis
Calculamos a importância dos sensores (variáveis) na previsão do risco de queimada. Isso nos ajuda a saber quais sensores são mais relevantes:

Fumaça → maior impacto

Temperatura

Umidade

Vento → menor impacto


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

# 1. Gerar dados sintéticos
np.random.seed(42)
n_amostras = 1000

# ─── Simular dados dos sensores ────────────────────────────────────────────────
temperatura = np.random.normal(loc=30, scale=5, size=n_amostras)
umidade = np.random.normal(loc=40, scale=10, size=n_amostras)
fumaca = np.random.normal(loc=100, scale=50, size=n_amostras)
vento = np.random.normal(loc=10, scale=3, size=n_amostras)

# ─── Criar rótulos de risco (0 = sem risco, 1 = com risco) ──────────────────────
risco_queimada = ((temperatura > 35) & (umidade < 30) & (fumaca > 120)).astype(int)

# Criar DataFrame com os dados
dados = pd.DataFrame({
    'temperatura': temperatura,
    'umidade': umidade,
    'fumaca': fumaca,
    'vento': vento,
    'risco_queimada': risco_queimada
})

# 2. Exploração dos Dados (EDA)
print("Primeiras linhas do conjunto de dados:")
display(dados.head(10))

print("\nEstatísticas descritivas:")
print(dados.describe())

# 2.1 Histograma das variáveis
fig, eixos = plt.subplots(2, 2, figsize=(12, 8))
eixos = eixos.flatten()
variaveis = ['temperatura', 'umidade', 'fumaca', 'vento']

for i, variavel in enumerate(variaveis):
    eixos[i].hist(dados[variavel], bins=30, edgecolor='black')
    eixos[i].set_title(f'Histograma de {variavel}')
    eixos[i].set_xlabel(variavel)
    eixos[i].set_ylabel('Frequência')

plt.tight_layout()
plt.show()

# 2.2 Contagem de classes
print("\nDistribuição de classes (risco de queimada):")
print(dados['risco_queimada'].value_counts())

# 2.3 Médias por classe
print("\nMédias das variáveis por classe (0 = sem risco, 1 = com risco):")
print(dados.groupby('risco_queimada').mean())

# 3. Separar variáveis independentes e alvo
X = dados[variaveis]
y = dados['risco_queimada']

# 3.1 Dividir dados em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. Treinar modelo Random Forest
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_treino, y_treino)

# 5. Avaliação no conjunto de teste
y_previsto = modelo.predict(X_teste)

print("\nMatriz de Confusão:")
print(confusion_matrix(y_teste, y_previsto))

print("\nRelatório de Classificação:")
print(classification_report(y_teste, y_previsto))

# 6. Validação cruzada (5-fold)
validacoes = cross_val_score(modelo, X, y, cv=5, scoring='accuracy')
print("Acurácia por validação cruzada (5-fold):", validacoes)
print("Acurácia média:", validacoes.mean())

# 7. Importância das variáveis
importancias = modelo.feature_importances_
importancia_variaveis = pd.Series(importancias, index=variaveis).sort_values(ascending=False)

print("\nImportância das variáveis (sensores):")
print(importancia_variaveis)

# 7.1 Gráfico de importância
plt.figure(figsize=(6, 4))
importancia_variaveis.plot(kind='bar', edgecolor='black')
plt.title('Importância dos Sensores na Previsão de Queimada')
plt.xlabel('Sensor')
plt.ylabel('Importância')
plt.tight_layout()
plt.show()

# 8. Conclusão
print("\n🔍 Conclusões:")
print("- Hipótese confirmada: alta temperatura, baixa umidade e alta fumaça se correlacionam com risco.")
print("- Modelo Random Forest mostrou boa performance nas métricas.")
print("- Importância dos sensores ajuda a priorizar dispositivos no sistema IoT.")


✅ Justificativa da Escolha do Modelo de Machine Learning e da Técnica Aplicada
Para o desenvolvimento do modelo preditivo de risco de queimadas com base em dados ambientais captados por sensores IoT, foi escolhida a técnica de classificação supervisionada com o algoritmo Random Forest Classifier. A justificativa para essa escolha se baseia em diversos critérios técnicos, conforme descrito a seguir:

🔎 1. Tipo de Problema: Classificação Binária
O problema em questão envolve prever se uma determinada combinação de variáveis ambientais representa risco de queimada (1) ou não representa risco (0). Isso caracteriza um problema típico de classificação binária supervisionada, em que o modelo aprende a partir de exemplos rotulados.

🌳 2. Escolha do Modelo: Random Forest
O modelo selecionado foi o Random Forest, que é um conjunto de árvores de decisão (ensemble). Ele apresenta várias vantagens em relação a modelos simples como regressão logística ou uma única árvore de decisão:

✅ Robustez a ruídos e overfitting: por combinar múltiplas árvores, o modelo reduz o risco de overfitting presente em árvores isoladas.

✅ Capacidade de lidar com relações não lineares: as interações entre temperatura, umidade e fumaça nem sempre seguem padrões lineares. O Random Forest trata bem essas complexidades.

✅ Importância das variáveis: o modelo oferece uma métrica clara de quais sensores (variáveis) são mais relevantes para o risco de queimada.

✅ Boa performance sem necessidade de muito ajuste de hiperparâmetros: ideal para projetos acadêmicos com tempo limitado.

⚙️ 3. Técnica Aplicada: Pipeline Supervisionado com Validação
O processo adotado seguiu as boas práticas de ciência de dados, com as seguintes etapas:

📊 Exploração e limpeza dos dados: análise estatística e visual para entendimento dos dados.

🧪 Divisão entre treino e teste (70/30), com estratificação da variável alvo.

🧠 Treinamento do modelo com Random Forest utilizando scikit-learn.

📈 Avaliação com métricas de classificação (precisão, recall, F1-score) e validação cruzada (cross-validation 5-fold) para testar a estabilidade do modelo.

📊 Análise da importância das variáveis para justificar tecnicamente o uso de sensores específicos.

🧠 4. Por que não foram escolhidos outros modelos?
❌ Regressão logística: assume relação linear entre variáveis, o que não se aplica bem ao problema.

❌ SVM: embora eficiente, exige mais ajuste e não é tão interpretável quanto Random Forest.

❌ Redes neurais: embora potentes, seriam desnecessárias para esse problema relativamente simples, além de menos interpretáveis e mais custosas computacionalmente.

🎯 Conclusão da Justificativa
O modelo Random Forest foi escolhido por sua eficiência, precisão e capacidade de interpretar variáveis ambientais complexas. Além disso, sua robustez e baixa exigência de ajuste fino o tornam ideal para aplicações práticas com sensores IoT, em que os dados podem apresentar ruídos e variações.

Essa escolha garante bom desempenho preditivo e facilidade de integração em sistemas de monitoramento em tempo real para prevenção de queimadas.

